In [2]:
using ForwardDiff, GenericSVD, LinearAlgebra

## Generalized SVD -- Jacobi Ensembles
$$ d([A;B]) = \prod_{i<j}|c_i^2 - c_j^2|\prod_i c_i^{m_1 - n} s_i^{m_2 - n}|H|^{(m_1+m_2-n)}(U'dU)^\wedge (V'd V)^\wedge(d\Theta)^\wedge (dH)^\wedge $$

In [4]:
function gsvd(AB,m₁)  # gsvd computation
    A = AB[1:m₁,:]
    B = AB[(m₁+1):end,:]
    n = size(A,2)
    U,σ,V = svd(A*pinv(B))
    U = U[:,1:n];  V = V[:,1:n]
    θ = acot.(σ[1:n])
    H = (cos.(θ)).\(U'A)    
    U,V,θ,H
end

gsvd (generic function with 1 method)

In [5]:
function gsvd_auto(A,B)
    m₁,n = size(A)
    AB=[A;B]
    JU,JV,Jθ,JH = (ForwardDiff.jacobian(x -> gsvd(x,m₁)[i], AB) for i=1:4)    
    U, σ, V = svd(A*pinv(B),thin=false)
    θ = acot.(σ[1:n])  
    Uk = kron(eye(n), U') * JU   
    Vk = kron(eye(n), V') * JV    
    rowsU = vec((tril(ones(m₁,n),-1))) .==1
    rowsV = vec((tril(ones(m₂,n),-1))) .==1    
    Uk = Uk[rowsU, :]; Vk = Vk[rowsV, :]   
    1/abs(det( [Jθ;JH;Uk;Vk] )) 
end

gsvd_auto (generic function with 1 method)

In [11]:
m₁,m₂,n = 5,4,4
A = randn(m₁,n); B = randn(m₂,n)
U,V,θ,H = gsvd([A;B],size(A,1))
c = cos.(θ); s = sin.(θ)
gsvd_auto(A,B),   #  Experiment (theory below)
abs(det(H)^(m₁+m₂-n) * prod([c[i]^2-c[j]^2 for i=1:n, j=1:n if i<j])) *prod(c)^(m₁-n) * prod(s)^(m₂-n) 

(96730.56500019798, 96730.56500019884)